In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from keras.applications import mobilenet
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
train_path = '/content/drive/MyDrive/cat_dog/data/train'
test_path = '/content/drive/MyDrive/cat_dog/data/test'
valid_path = '/content/drive/MyDrive/cat_dog/data/valid'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), batch_size=30)
valid_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    valid_path, target_size=(224,224), batch_size=30)
test_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), batch_size=30)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 2 classes.


In [ ]:
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3))
base_model.summary()

58889256/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

In [ ]:
last_layer = base_model.get_layer('block5_pool')

In [ ]:
last_output = last_layer.output

In [ ]:
x = Flatten()(last_output)

In [ ]:
x = Dense(64, activation='relu', name='FC_2')(x)
x = BatchNormalization()(x) # 배치 단위로 데터를 정규화 함으로서 안정적인 학습이 이루어짐
x = Dropout(0.5)(x) # 과적합을 줄이기 위함

In [ ]:
x = Dense(2, activation='softmax', name='softmax')(x)
# 개/고양이 분류기

In [ ]:
new_model = Model(inputs=base_model.input, outputs=x)

In [ ]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
new_model.compile(Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
new_model.fit_generator(train_batches, steps_per_epoch=4,
                   validation_data=valid_batches, validation_steps=2, epochs=20, verbose=2)

<ipython-input-14-6c0f84f949e8>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  new_model.fit_generator(train_batches, steps_per_epoch=4,


Epoch 1/20
4/4 - 282s - loss: 1.1469 - accuracy: 0.5625 - val_loss: 1.5142 - val_accuracy: 0.7500 - 282s/epoch - 70s/step
Epoch 2/20
4/4 - 288s - loss: 0.6456 - accuracy: 0.7500 - val_loss: 4.8237 - val_accuracy: 0.4667 - 288s/epoch - 72s/step
Epoch 3/20
4/4 - 261s - loss: 0.9298 - accuracy: 0.5714 - val_loss: 0.9635 - val_accuracy: 0.4833 - 261s/epoch - 65s/step
Epoch 4/20
4/4 - 264s - loss: 0.8912 - accuracy: 0.6161 - val_loss: 9.2345 - val_accuracy: 0.4667 - 264s/epoch - 66s/step
Epoch 5/20
4/4 - 274s - loss: 0.9092 - accuracy: 0.6500 - val_loss: 12.9226 - val_accuracy: 0.5500 - 274s/epoch - 69s/step
Epoch 6/20
4/4 - 282s - loss: 0.8868 - accuracy: 0.6083 - val_loss: 15.9453 - val_accuracy: 0.4500 - 282s/epoch - 70s/step
Epoch 7/20


KeyboardInterrupt: 

In [ ]:
def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets

In [ ]:
from sklearn.datasets import load_files
from tensorflow.keras.utils import np_utils
import numpy as np
test_files, test_targets = load_dataset("/content/drive/MyDrive/cat_dog/data/test")

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tqdm import tqdm
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)
def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)
test_tensors = preprocess_input(paths_to_tensor(test_files))

In [ ]:
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets)))

In [ ]:
# evaluate and print test accuracy
score = new_model.evaluate(test_tensors, test_targets)
print('\n', 'Test accuracy:', score[1])